In [1]:
import pandas as pd
import numpy as np

def to_month_id(year, month):
    return (year - 1996) * 12 + month - 6

In [7]:
dfFeature = pd.read_csv("./lstm_2004_40.csv")
dfFeature["return"] = np.sign(dfFeature["return"])
features = dfFeature.drop(columns=['date', 'year', 'month', 'month_id', 'QAId']).as_matrix()
print(features)
dfFeature[dfFeature.return == 0] = dfFeature[dfFeature == 0]

[[ 0.13141069  0.34014825 -0.12098358 ... -0.152407    6.13
   1.        ]
 [ 1.94886748  1.65489218 -1.14691204 ... -0.152407    6.13
  -1.        ]
 [ 0.65251814 -1.24161551 -1.35219889 ... -0.152407    6.13
   1.        ]
 ...
 [ 0.56471281 -0.33312133  1.27570117 ...  0.03655525  4.9
   1.        ]
 [ 0.00935546 -1.10331101  0.96957883 ...  0.03655525  4.9
   1.        ]
 [-0.53732841  0.32135129 -1.23000641 ...  0.03655525  4.9
  -1.        ]]


In [10]:
from sklearn.preprocessing import MinMaxScaler
normalizedFeatures = MinMaxScaler().fit_transform(features)
print(normalizedFeatures.shape)
print(normalizedFeatures)

(73005, 54)
[[0.55852562 0.57368947 0.5293905  ... 0.0178837  0.30909091 1.        ]
 [0.9334597  0.8423187  0.3997125  ... 0.0178837  0.30909091 0.        ]
 [0.66602801 0.2505026  0.37376411 ... 0.0178837  0.30909091 1.        ]
 ...
 [0.64791412 0.43612665 0.70593232 ... 0.52513808 0.08545455 1.        ]
 [0.53334612 0.27876105 0.66723827 ... 0.52513808 0.08545455 1.        ]
 [0.42056742 0.56984887 0.38920932 ... 0.52513808 0.08545455 0.        ]]


In [35]:
dfFeature.groupby("return").count().head(5)

,month_id,QAId,us_R3_CREFb_PutCallParity,us_R3_CREFb_VolumeRatio,50To200PrcRatio,LogMktCap,14DayRSI,ShortIntTrdVol,SalesToInvCap,RelPrStr_12M,...,GovSpending,OilLogRet12M,PMI,SPXLogRet12M,TermSpread,USDLogRet12M,Unemployment,date,year,month
return,,,,,,,,,,,,,,,,,,,,,
-1.0,30938,30938,30938,30938,30938,30938,30938,30938,30938,30938,...,30938,30938,30938,30938,30938,30938,30938,30938,30938,30938
0.0,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
1.0,42048,42048,42048,42048,42048,42048,42048,42048,42048,42048,...,42048,42048,42048,42048,42048,42048,42048,42048,42048,42048


In [11]:
dfTarget = pd.read_csv("./return_2004_40.csv")
dfTarget["return"] = np.sign(dfTarget["return"])
targets = MinMaxScaler().fit_transform(dfTarget[dfTarget.month_id >= 115].drop(columns=['month_id', 'QAId']).as_matrix())
y = np.asarray(np.split(targets, len(targets) / 465))
print(y.shape)
print(y)

(134, 465, 1)
[[[1.]
  [1.]
  [0.]
  ...
  [0.]
  [1.]
  [1.]]

 [[1.]
  [1.]
  [1.]
  ...
  [1.]
  [1.]
  [0.]]

 [[0.]
  [1.]
  [1.]
  ...
  [0.]
  [1.]
  [0.]]

 ...

 [[0.]
  [0.]
  [1.]
  ...
  [0.]
  [0.]
  [1.]]

 [[1.]
  [1.]
  [1.]
  ...
  [1.]
  [1.]
  [0.]]

 [[1.]
  [1.]
  [1.]
  ...
  [1.]
  [0.]
  [1.]]]


In [12]:
from keras.preprocessing.sequence import TimeseriesGenerator

splittedFeature = np.asarray(np.split(normalizedFeatures, len(normalizedFeatures) / 465)).swapaxes(0, 1)
print(splittedFeature.shape)

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(465, 157, 54)


In [14]:
from keras.preprocessing.sequence import TimeseriesGenerator

x_table = np.zeros((465, 134, 24, 54))
y_mock = np.zeros((157, 1))

for stock in range(465):
  x_windowed = TimeseriesGenerator(splittedFeature[stock], y_mock, length=24, batch_size=1)
  x_table[stock] = x_windowed[0][0]

x = x_table.swapaxes(0, 1)
print(x.shape)
print(x)

(134, 465, 24, 54)
[[[[0.55852562 0.57368947 0.5293905  ... 0.0178837  0.30909091
    1.        ]
   [0.56746299 0.57834781 0.5460191  ... 0.0178837  0.30909091
    1.        ]
   [0.55408858 0.62572145 0.59148273 ... 0.06709876 0.25454545
    0.        ]
   ...
   [0.51619134 0.56942131 0.63712194 ... 0.60504974 0.12181818
    1.        ]
   [0.50774967 0.57765073 0.58663953 ... 0.72005515 0.12181818
    1.        ]
   [0.50192502 0.5071093  0.59781561 ... 0.72005515 0.09818182
    1.        ]]

  [[0.9334597  0.8423187  0.3997125  ... 0.0178837  0.30909091
    0.        ]
   [0.92623195 0.82174114 0.34316862 ... 0.0178837  0.30909091
    1.        ]
   [0.95079918 0.84544858 0.41779343 ... 0.06709876 0.25454545
    1.        ]
   ...
   [0.64656372 0.66420653 0.43412555 ... 0.60504974 0.12181818
    1.        ]
   [0.61442513 0.57751688 0.34555234 ... 0.72005515 0.12181818
    0.        ]
   [0.64253599 0.57576546 0.29878152 ... 0.72005515 0.09818182
    1.        ]]

  [[0.66602801 

In [27]:
from sklearn.model_selection import PredefinedSplit

x_train = x[0:108].reshape(108 * 465, 24, 54)
y_train = y[0:108].reshape(108 * 465, 1)
x_test = x[108:134].reshape(26 * 465, 24, 54)
y_test = y[108:134].reshape(26 * 465, 1)


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

ps = PredefinedSplit(np.append(np.negative(np.ones(60 * 465)), np.zeros(24 * 465))).split(x_train)

for train_ids, test_ids in ps:
  print(str(train_ids) + ", " + str(test_ids))

(50220, 24, 54)
(50220, 1)
(12090, 24, 54)
(12090, 1)
[    0     1     2 ... 27897 27898 27899], [27900 27901 27902 ... 39057 39058 39059]


In [36]:
t = np.array((x_train, y_train, x_test, y_test))
np.save("./dataset.npy", t)

In [32]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Flatten
from keras.regularizers import l1_l2, l2
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

np.random.seed(4103)

def create_model(*param):
    model = Sequential()
    model.add(LSTM(units=32, input_shape=(24, 54), return_sequences=True, kernel_regularizer=l1_l2(0.01), unroll=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=32, return_sequences=True, kernel_regularizer=l1_l2(0.01), unroll=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=32, return_sequences=False, kernel_regularizer=l1_l2(0.01), unroll=True))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation="relu", kernel_regularizer=l1_l2(0.01)))
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.01, decay=0.0), metrics=['accuracy'], *param)
    return model

param_search = {
    'activation': ['tanh', 'relu']
}

In [33]:
create_model().fit(x_train, y_train, batch_size=465, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=465)
print(score)


Epoch 1/100
50220/50220 [==============================] - 35s 705us/step - loss: 3.8129 - acc: 0.5108
Epoch 2/100
50220/50220 [==============================] - 14s 272us/step - loss: 1.4563 - acc: 0.5595
Epoch 3/100
50220/50220 [==============================] - 16s 328us/step - loss: 1.4475 - acc: 0.5743
Epoch 4/100
50220/50220 [==============================] - 17s 331us/step - loss: 1.4435 - acc: 0.5758
Epoch 5/100
50220/50220 [==============================] - 12s 236us/step - loss: 1.4511 - acc: 0.5758
Epoch 6/100
50220/50220 [==============================] - 11s 224us/step - loss: 1.4424 - acc: 0.5758
Epoch 7/100
50220/50220 [==============================] - 12s 235us/step - loss: 1.4424 - acc: 0.5758
Epoch 8/100
50220/50220 [==============================] - 12s 232us/step - loss: 1.4424 - acc: 0.5758
Epoch 9/100
50220/50220 [==============================] - 14s 271us/step - loss: 1.4425 - acc: 0.5758
Epoch 10/100
50220/50220 [==============================] - 13s 260us/ste

KeyboardInterrupt: 

In [ ]:
gscv = GridSearchCV(KerasClassifier(build_fn=create_model), \
                    cv=ps, \
                    param_grid=param_search, \
                    scoring='accuracy')

gscv.fit(x_train, y_train, batch_size=465, epochs=10)

# print("Best parameters set found on training set:")
# print(gscv.best_params_)

# means_valid = gscv.cv_results_['mean_test_score']
# stds_valid = gscv.cv_results_['std_test_score']
# means_train = gscv.cv_results_['mean_train_score']

# print("Grid scores:")
# for mean_valid, std_valid, mean_train, params in zip(means_valid, stds_valid, means_train, gscv.cv_results_['params']):
#     print("Validation: %0.3f (+/-%0.03f), Training: %0.3f  for %r" % (mean_valid, std_valid, mean_train, params))

In [ ]:
print(np.random.randint(2, size=(465 * 26, 1)))

In [ ]:
input_shape = (465, 24, 221)

x_train = np.random.random((465 * 108, 24, 221))
y_train = np.random.randint(2, size=(465 * 108, 1))
print(x_train.shape)

x_test = np.random.random((465 * 26, 24, 221))
y_test = np.random.randint(2, size=(465 * 26, 1))
print(x_test.shape)

model = Sequential()
model.add(LSTM(units = 1, \
               input_shape=(24, 221), \
               activation='tanh', \
               recurrent_activation='hard_sigmoid', \
               use_bias=True, \
               kernel_initializer='glorot_uniform', \
               recurrent_initializer='orthogonal', \
               bias_initializer='zeros', \
               unit_forget_bias=True, \
               kernel_regularizer=l1_l2(0.01), \
               bias_regularizer=l1_l2(0.01), \
               activity_regularizer=l1_l2(0.01), \
               dropout=0.1, \
               recurrent_dropout=0.1, \
               unroll=True))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=465, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=465)
print(score)